# 1.회원가입 & 로그인 만들기

회원가입 기능과 로그인을 하기 위해 SQL에 대해 먼저 이해하자.

SQL은 데이터를 다루기 위한 표준 언어이다.

사용자가 회원가입을 하면 그 데이터를 처리하기 위해 SQL을 사용해야한다.

이때 우리는 flask에서 SQL을 몰라도 SQLAlchemy를 통해 DB를 사용 할 수 있게 해준다.

    - from flask_sqlalchemy import SQLAlchemy 을 통해 import 해오자.

In [ ]:
from flask import Flask, render_template, request, redirect, session, url_for
from flask_sqlalchemy import SQLAlchemy

(1)Flask(\__name__)을 통해 서버를 app으로 열자.

(2)SQLAlchmey(app)을 통해 직접 연 서버를 SQL과 db로 연결하자.

In [ ]:
app = Flask(__name__)
db = SQLAlchemy(app)

## (2)회원가입 정보 입력 class 만들기

User class는 db = SQLAlchemy()의 model을 상속받는다. 

즉 인자로 들어오는 값들을 테이블로 변환해주는 역할을 한다.

이때 db에 저장되는 값은 각열에 id(사용자 번호), username(사용자 아이디), password(비밀번호)

    - unique = True

id 의 중복을 방지해준다.

    - nullable = False

빈칸 입력을 방지해준다.

    - primary_key = True

사용자를 구별하기위해 id를 중복 방지하여 빠르게 찾을 수 있다.

In [ ]:
class User(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    username = db.Column(db.String(20), unique = True, nullable = False)
    password = db.Column(db.String(100), nullable = False)

## (3)만들어진 db 테이블을 저장하기

In [ ]:
with app.app_context():
    db.create_all()

## (4) 회원가입 등록하기

    -@app.route('/register', methods=["GET", "POST"])

사용자가 /register 페이지에 접속(get)하거나 가입 양식을 제출(post)할때 호출되는 함수이다.

    -if request.method == "POST":

사용자가 가입 폼을 제출하게 되면 실행되는 함수이다. 이때 사용자가 입력한(request)를 받아 각각 HTML의 <from>에서 전송된 데이터 저장소(requst.form)에 저장한다.


    -if User.query.filter_by(username = username).first():

기존의 sql 데이터 베이스에서 같은 아이디가 있는지 확인한다.

검사를 진행 후 User class의 새로운 사용자 인스턴스를 생성해서 db(데이터베이스)에 저장할 대기열 추가한후 commit을 통해 반영한다. 

**현재는 비밀번호가 평문 저장되어있는 상태. hash를 통해 암호화 필요

    -return redirect(url_for('login'))

회원가입을 성고하면 /login 페이지로 이동한다. url_for은 @app.route()로 연결되는 주소를 자동으로 찾아준다.





In [ ]:
@app.route('/register', methods = ["GET", "POST"])
def register():
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']

        if User.query.filter_by(username = username).first():
            return "이미 존재하는 아이디입니다"
        
        new_user = User(username = username, password = password)
        db.session.add(new_user)
        db.session.commit()

        return redirect(url_for('login'))
    return render_template("register.html")